# Load Dataset

In [1]:
test_images_path = r'/home/daniel/Git/Hot-Topics-in-NLP/notebooks/homeworks/homework_4/test_images_resized'
train_images_path = r'/home/daniel/Git/Hot-Topics-in-NLP/notebooks/homeworks/homework_4/train_trial_images/train_v1'

# CLIP Pretrained

In [2]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image.show()

Given an image, the CLIPVisionModel returns the embedding of the respective text description

In [6]:
from transformers import AutoProcessor, CLIPVisionModel

model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled CLS states

# Access the image embeddings
image_embedding = outputs.last_hidden_state 
image_embedding

tensor([[[-0.1391,  0.1016, -0.2362,  ...,  0.1568,  0.2143,  0.2181],
         [ 0.1976,  0.4183, -0.8494,  ...,  0.1030,  1.4616,  0.4321],
         [ 0.1630,  0.2004, -1.0608,  ..., -0.0640,  1.3778,  0.5354],
         ...,
         [ 0.0552, -0.0113, -0.6040,  ...,  0.2935,  1.0303, -0.0316],
         [ 0.1673,  0.0491, -0.3833,  ...,  0.3797,  0.9275, -0.0212],
         [ 0.1937,  0.2567, -0.4325,  ...,  0.2957,  1.2151, -0.0417]]],
       grad_fn=<AddBackward0>)

Given some text, the CLIPTextModel returns an image corresponding to the embedding of the provided text 

In [7]:
# Abracadabra

For each row of the dataset we have:
1. the word
2. a minimal context
3. a list of 10 images, among which only one refers to the real word context

We can use the pretrained version of CLIP this way:
1. prompt clip injecting the word and the respective context
2. compute the text embedding for the prompt
3. compute the image embeddings for each of the 10 images
4. measure the cosine similarity between the text embedding and each of the 10 image embeddings,
    ranking them

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def min_cosine(target_vector, vector_list):
    similarities = [cosine_similarity([target_vector], [vector])[0, 0] for vector in vector_list]
    best_index = np.argmin(similarities)
    best_vector = vector_list[best_index]
    return best_vector, similarities[best_index]

def cosine_similarity_key(vector1, vector2):
    return cosine_similarity([vector1], [vector2])[0, 0]

def rank_cosine(target_vector, vector_list):
    sorted_vectors = sorted(vector_list, key=lambda vector: cosine_similarity_key(target_vector, vector), reverse=True)
    return sorted_vectors

# Example usage:
target_vector = np.array([1, 2, 3])  # Replace with your actual target vector
vector_list = [np.array([4, 5, 6]), np.array([7, 8, 9]), np.array([10, 11, 12])]  # Replace with your actual list of vectors

best_vector, min_similarity = min_cosine(target_vector, vector_list)

print(f"The vector with the smallest cosine similarity is: {best_vector}")
print(f"Cosine similarity value: {min_similarity}")

In [ ]:
predictions = []
for test_entry, gold_entry in zip(test_data, test_goal):
    word, minimal_context, image_list = test_entry
    prompt = f'This is a picture of {word} in the context of {minimal_context}'

    prompt_embedding = ...
    images_embeddings = []
    for image in image_list:
        image_embedding = ...
        images_embeddings.append(image_embedding)

    ranked_embeddings = rank_cosine ...

    predictions.append(ranked_embeddings)

# TODO

1. Use wordnet to inject synonyms and make data augmentation
2. Train your own CLIP-like model